In [3]:
project_path='/content/drive/MyDrive/CryptoFutureX'

In [4]:
!pip install  transformers

     |████████████████████████████████| 1.9MB 16.9MB/s 
     |████████████████████████████████| 890kB 49.5MB/s 
     |████████████████████████████████| 3.2MB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9009da585f02ea4b30cd461db48bbcc69f542c66d20bbfac2e2fc8c3e5c32f73
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn import metrics
from tqdm import tqdm
import numpy as np
import re

In [6]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [7]:
df_train = pd.read_csv("/content/drive/MyDrive/SentimentAnalysis-master/SentimentAnalysis-master/data/traindata_clean.csv")[["tweet","target"]]

In [8]:
df_train.head()

,tweet,target
0,is upset that he can't update his facebook by ...,0
1,entity i dived many times for the ball. manage...,0
2,my whole body feels itchy and like its on fire,0
3,"entity no, it's not behaving at all. i'm mad. ...",0
4,entity not the whole crew,0


In [9]:
def remove_URL(tweet):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',tweet)

def remove_numbers(tweet):
    tweet = ''.join([i for i in tweet if not i.isdigit()])         
    return tweet

def remove_html(tweet):
    html=re.compile(r'<.*?>')
    return html.sub(r'',tweet)

def remove_username(tweet):
    url = re.compile(r'@[A-Za-z0-9_]+')
    return url.sub(r'',tweet)

In [10]:
def pre_process_tweet(tweet):
    tweet = remove_URL(tweet)
    tweet = remove_numbers(tweet)
    tweet = remove_html(tweet)
    tweet = remove_username(tweet)
    return " ".join(tweet.split())


In [11]:
# longest tweet length
max(df_train['tweet'].apply(len))

367

In [12]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 160
BATCH_SIZE = 16
EPOCHS = 1
LEARNING_RATE = 1e-05


#BERT_PATH = '/distillbert-huggingface-model/'

MODEL_PATH = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')



In [13]:
class tweet_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        tweet = str(self.data.tweet[index])
        tweet = pre_process_tweet(tweet)
        inputs = self.tokenizer.encode_plus(
            tweet,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.target[index], dtype=torch.float)
        }
        
    def __len__(self):
        return len(self.data)

In [14]:
# Creating the dataset and dataloader for the neural network

train_size = 0.85
train_dataset=df_train.sample(frac=train_size,random_state=200).reset_index(drop=True)
valid_dataset=df_train.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df_train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALID Dataset: {}".format(valid_dataset.shape))

training_set = tweet_Dataset(train_dataset, tokenizer, MAX_LEN)
validation_set = tweet_Dataset(valid_dataset, tokenizer, MAX_LEN)


FULL Dataset: (1529862, 2)
TRAIN Dataset: (1300383, 2)
VALID Dataset: (229479, 2)


In [15]:
training_set[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'ids': tensor([  101,  2012,  1052,  2546, 11132,  2015,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [16]:
train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

train_dl = DataLoader(training_set, **train_params)
valid_dl = DataLoader(validation_set, **valid_params)


**Bert  Encoder(pooled output) -> Dropout -> Linear layer -> Final Output**

In [17]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.distill_bert = transformers.DistilBertModel.from_pretrained(MODEL_PATH )
        self.drop = torch.nn.Dropout(0.3)
        self.out = torch.nn.Linear(768, 1)
    
    def forward(self, ids, mask):
        distilbert_output = self.distill_bert(ids, mask)
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        output_1 = self.drop(pooled_output)
        output = self.out(output_1)
        return output

In [18]:
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (distill_bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Line

In [19]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


**EVALUTION FUNCTION**

In [20]:
def eval_fn(data_loader, model):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
        fin_outputs = np.array(fin_outputs) >= 0.5
        f1 = metrics.f1_score(fin_targets, fin_outputs)
    return f1

**TRAINING FUNCTION**

In [21]:
def fit(num_epochs, model, loss_fn, opt, train_dl, valid_dl):
    
    for epoch in range(num_epochs):
        model.train()
        for _,data in enumerate(train_dl, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask).squeeze()
            loss = loss_fn(outputs, targets)
            loss.backward()
            opt.step()
            opt.zero_grad()

        valid_acc = eval_fn(valid_dl, model)
        print('Epoch [{}/{}], Train Loss: {:.4f} and Validation f1 {:.4f}'.format(epoch+1, num_epochs, loss.item(),valid_acc))

**Training for seven epochs** 

In [ ]:
fit(7, model, loss_fn, optimizer, train_dl,valid_dl)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
